In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD

In [3]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow_addons as tfa

In [4]:
#Import and split data
#data = pd.read_csv('../input/voice.csv')
data_train = pd.read_csv('train_features.csv')
data_test = pd.read_csv('test_features.csv')
data_train_target_ns = pd.read_csv('train_targets_nonscored.csv')
data_train_target_s = pd.read_csv('train_targets_scored.csv')
ss = pd.read_csv('sample_submission.csv')


#Preprocess
#Categorize data


def preprocess(df):
    df = df.copy()
    # transform treatment group
    # transform vehicule group
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    
    # transform D1 and D2 group
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    
    # # transform 
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 1, 72: 2})
    del df['sig_id']
    return df



train = preprocess(data_train)
test = preprocess(data_test)
del data_train_target_s['sig_id']
top_features = list(range(1, len(train.columns)))

In [5]:
train.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,0,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,0,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,0,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,0,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,0,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [6]:
def create_model(num_columns):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(num_columns))
        
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(510, activation='relu')))
        
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.35))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(250, activation='relu')))

    #============ Final Layer =================
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid")))

    model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 700), 
                  loss='binary_crossentropy', metrics=['accuracy']
                  )
    return model

# model.fit(train_dataset, epochs=10, batch_size=128)

In [7]:
def metric(y_true, y_pred):
    metrics = []
    for _target in data_train_target_s.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import log_loss
N_STARTS = 9

res = data_train_target_s.copy()
ss.loc[:, data_train_target_s.columns] = 0
res.loc[:, data_train_target_s.columns] = 0

top_feats = list(range(0, 785))
historys = dict()

#print (ss.loc[:, data_train_target_s.columns])

tf.random.set_seed(43)
for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed*3, shuffle=True).split(data_train_target_s, data_train_target_s)):
        print(f"======{data_train_target_s.values[tr].shape}========{data_train_target_s.values[te].shape}=====")
        print(f'Seed: {seed} => Fold: {n}')
        
        model = create_model(len(top_feats))
        
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_lr=1e-5, patience=3, verbose=1, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 1, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        early = EarlyStopping(monitor="val_loss", mode="min", restore_best_weights=True, patience= 5, verbose = 1)
        
        history = model.fit(train.values[tr][:, top_feats], data_train_target_s.values[tr], validation_data=(train.values[te][:, top_feats], data_train_target_s.values[te]),
                  epochs=100, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt, early], verbose=2)
            
        historys[f'history_{seed+1}'] = history
        print("Model History Saved.")
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        ss.loc[:, data_train_target_s.columns] += test_predict
        res.loc[te, data_train_target_s.columns] += val_predict
        
        print(f'OOF Metric For SEED {seed} => FOLD {n} : {metric(data_train_target_s.loc[te, data_train_target_s.columns], pd.DataFrame(val_predict, columns=data_train_target_s.columns))}')
        print('+-' * 10)
    
ss.loc[:, data_train_target_s.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train_target_s.columns] /= N_STARTS

======(20412, 206)========(3402, 206)=====
Seed: 0 => Fold: 0
Epoch 1/100

Epoch 00001: val_loss improved from inf to 0.11610, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.4827 - accuracy: 0.0160 - val_loss: 0.1161 - val_accuracy: 0.0121
Epoch 2/100

Epoch 00002: val_loss improved from 0.11610 to 0.03484, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0616 - accuracy: 0.0283 - val_loss: 0.0348 - val_accuracy: 0.0362
Epoch 3/100

Epoch 00003: val_loss improved from 0.03484 to 0.02385, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0286 - accuracy: 0.0469 - val_loss: 0.0239 - val_accuracy: 0.0720
Epoch 4/100

Epoch 00004: val_loss improved from 0.02385 to 0.02086, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0232 - accuracy: 0.0540 - val_loss: 0.0209 - val_accuracy: 0.0438
Epoch 5/100

Epoch 00005: val_loss improved from 0.02086 to 0.01986, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0215 - accuracy: 0.0583 - val_los


Epoch 00042: val_loss improved from 0.01518 to 0.01518, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0140 - accuracy: 0.1549 - val_loss: 0.0152 - val_accuracy: 0.1329
Epoch 43/100

Epoch 00043: val_loss improved from 0.01518 to 0.01518, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0140 - accuracy: 0.1572 - val_loss: 0.0152 - val_accuracy: 0.1320
Epoch 44/100

Epoch 00044: val_loss improved from 0.01518 to 0.01517, saving model to repeat:0_Fold:0.hdf5
160/160 - 2s - loss: 0.0140 - accuracy: 0.1597 - val_loss: 0.0152 - val_accuracy: 0.1320
Epoch 45/100

Epoch 00045: val_loss did not improve from 0.01517
160/160 - 2s - loss: 0.0140 - accuracy: 0.1589 - val_loss: 0.0152 - val_accuracy: 0.1320
Epoch 46/100

Epoch 00046: val_loss did not improve from 0.01517
160/160 - 2s - loss: 0.0139 - accuracy: 0.1564 - val_loss: 0.0152 - val_accuracy: 0.1314
Epoch 47/100

Epoch 00047: val_loss did not improve from 0.01517
160/160 - 2s - loss: 0.0140 - accuracy: 0.1553 - val

Epoch 36/100

Epoch 00036: val_loss improved from 0.01490 to 0.01489, saving model to repeat:0_Fold:1.hdf5
160/160 - 2s - loss: 0.0140 - accuracy: 0.1557 - val_loss: 0.0149 - val_accuracy: 0.1387
Epoch 37/100

Epoch 00037: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00037: val_loss improved from 0.01489 to 0.01487, saving model to repeat:0_Fold:1.hdf5
160/160 - 2s - loss: 0.0140 - accuracy: 0.1571 - val_loss: 0.0149 - val_accuracy: 0.1384
Epoch 38/100

Epoch 00038: val_loss improved from 0.01487 to 0.01487, saving model to repeat:0_Fold:1.hdf5
160/160 - 2s - loss: 0.0139 - accuracy: 0.1587 - val_loss: 0.0149 - val_accuracy: 0.1384
Epoch 39/100

Epoch 00039: val_loss improved from 0.01487 to 0.01487, saving model to repeat:0_Fold:1.hdf5
160/160 - 2s - loss: 0.0139 - accuracy: 0.1578 - val_loss: 0.0149 - val_accuracy: 0.1387
Epoch 40/100

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1e-05.

Epoch 00040: val_loss improved from 0.01487 to 0.01487,

Epoch 30/100

Epoch 00030: val_loss improved from 0.01554 to 0.01543, saving model to repeat:0_Fold:2.hdf5
160/160 - 2s - loss: 0.0146 - accuracy: 0.1439 - val_loss: 0.0154 - val_accuracy: 0.1285
Epoch 31/100

Epoch 00031: val_loss improved from 0.01543 to 0.01539, saving model to repeat:0_Fold:2.hdf5
160/160 - 2s - loss: 0.0146 - accuracy: 0.1435 - val_loss: 0.0154 - val_accuracy: 0.1270
Epoch 32/100

Epoch 00032: val_loss improved from 0.01539 to 0.01536, saving model to repeat:0_Fold:2.hdf5
160/160 - 2s - loss: 0.0146 - accuracy: 0.1429 - val_loss: 0.0154 - val_accuracy: 0.1282
Epoch 33/100

Epoch 00033: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00033: val_loss improved from 0.01536 to 0.01535, saving model to repeat:0_Fold:2.hdf5
160/160 - 2s - loss: 0.0145 - accuracy: 0.1437 - val_loss: 0.0154 - val_accuracy: 0.1261
Epoch 34/100

Epoch 00034: val_loss improved from 0.01535 to 0.01534, saving model to repeat:0_Fold:2.hdf5
160/160 - 2s - loss: 0.0145

Epoch 27/100

Epoch 00027: val_loss improved from 0.01551 to 0.01541, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0154 - accuracy: 0.1298 - val_loss: 0.0154 - val_accuracy: 0.1173
Epoch 28/100

Epoch 00028: val_loss did not improve from 0.01541
160/160 - 2s - loss: 0.0152 - accuracy: 0.1323 - val_loss: 0.0155 - val_accuracy: 0.1220
Epoch 29/100

Epoch 00029: val_loss improved from 0.01541 to 0.01534, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0151 - accuracy: 0.1351 - val_loss: 0.0153 - val_accuracy: 0.1208
Epoch 30/100

Epoch 00030: val_loss improved from 0.01534 to 0.01529, saving model to repeat:0_Fold:3.hdf5
160/160 - 2s - loss: 0.0150 - accuracy: 0.1373 - val_loss: 0.0153 - val_accuracy: 0.1226
Epoch 31/100

Epoch 00031: val_loss did not improve from 0.01529
160/160 - 2s - loss: 0.0149 - accuracy: 0.1434 - val_loss: 0.0153 - val_accuracy: 0.1208
Epoch 32/100

Epoch 00032: val_loss did not improve from 0.01529
160/160 - 2s - loss: 0.0148 - accuracy:

...

Epoch 37/100

Epoch 00037: ReduceLROnPlateau reducing learning rate to 1e-05.

Epoch 00037: val_loss improved from 0.01525 to 0.01524, saving model to repeat:3_Fold:1.hdf5
160/160 - 2s - loss: 0.0143 - accuracy: 0.1524 - val_loss: 0.0152 - val_accuracy: 0.1337
Epoch 38/100

Epoch 00038: val_loss did not improve from 0.01524
160/160 - 2s - loss: 0.0142 - accuracy: 0.1515 - val_loss: 0.0152 - val_accuracy: 0.1335
Epoch 39/100

Epoch 00039: val_loss improved from 0.01524 to 0.01524, saving model to repeat:3_Fold:1.hdf5
160/160 - 2s - loss: 0.0142 - accuracy: 0.1512 - val_loss: 0.0152 - val_accuracy: 0.1340
Epoch 40/100

Epoch 00040: val_loss did not improve from 0.01524
160/160 - 2s - loss: 0.0142 - accuracy: 0.1529 - val_loss: 0.0152 - val_accuracy: 0.1335
Epoch 41/100

Epoch 00041: val_loss did not improve from 0.01524
160/160 - 2s - loss: 0.0142 - accuracy: 0.1511 - val_loss: 0.0152 - val_accuracy: 0.1337
Epoch 42/100

Epoch 00042: val_loss did not improve from 0.01524
160/160 - 2s - l

Epoch 32/100

Epoch 00032: val_loss improved from 0.01529 to 0.01528, saving model to repeat:3_Fold:2.hdf5
160/160 - 2s - loss: 0.0144 - accuracy: 0.1483 - val_loss: 0.0153 - val_accuracy: 0.1264
Epoch 33/100

Epoch 00033: val_loss improved from 0.01528 to 0.01527, saving model to repeat:3_Fold:2.hdf5
160/160 - 2s - loss: 0.0144 - accuracy: 0.1488 - val_loss: 0.0153 - val_accuracy: 0.1255
Epoch 34/100

Epoch 00034: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.

Epoch 00034: val_loss improved from 0.01527 to 0.01526, saving model to repeat:3_Fold:2.hdf5
160/160 - 2s - loss: 0.0144 - accuracy: 0.1495 - val_loss: 0.0153 - val_accuracy: 0.1258
Epoch 35/100

Epoch 00035: val_loss improved from 0.01526 to 0.01526, saving model to repeat:3_Fold:2.hdf5
160/160 - 2s - loss: 0.0142 - accuracy: 0.1490 - val_loss: 0.0153 - val_accuracy: 0.1261
Epoch 36/100

Epoch 00036: val_loss improved from 0.01526 to 0.01526, saving model to repeat:3_Fold:2.hdf5
160/160 - 2s - loss: 0.0143

Epoch 28/100

Epoch 00028: val_loss improved from 0.01580 to 0.01575, saving model to repeat:3_Fold:3.hdf5
160/160 - 2s - loss: 0.0149 - accuracy: 0.1387 - val_loss: 0.0158 - val_accuracy: 0.1211
Epoch 29/100

Epoch 00029: val_loss did not improve from 0.01575
160/160 - 2s - loss: 0.0147 - accuracy: 0.1428 - val_loss: 0.0158 - val_accuracy: 0.1214
Epoch 30/100

Epoch 00030: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00030: val_loss improved from 0.01575 to 0.01570, saving model to repeat:3_Fold:3.hdf5
160/160 - 2s - loss: 0.0147 - accuracy: 0.1453 - val_loss: 0.0157 - val_accuracy: 0.1238
Epoch 31/100

Epoch 00031: val_loss improved from 0.01570 to 0.01562, saving model to repeat:3_Fold:3.hdf5
160/160 - 2s - loss: 0.0143 - accuracy: 0.1497 - val_loss: 0.0156 - val_accuracy: 0.1261
Epoch 32/100

Epoch 00032: val_loss improved from 0.01562 to 0.01560, saving model to repeat:3_Fold:3.hdf5
160/160 - 2s - loss: 0.0142 - accuracy: 0.1494 - val_loss: 0.0156 - v

Epoch 25/100

Epoch 00025: val_loss improved from 0.01553 to 0.01548, saving model to repeat:3_Fold:4.hdf5
160/160 - 2s - loss: 0.0155 - accuracy: 0.1267 - val_loss: 0.0155 - val_accuracy: 0.1235
Epoch 26/100

Epoch 00026: val_loss improved from 0.01548 to 0.01542, saving model to repeat:3_Fold:4.hdf5
160/160 - 2s - loss: 0.0154 - accuracy: 0.1274 - val_loss: 0.0154 - val_accuracy: 0.1243
Epoch 27/100

Epoch 00027: val_loss did not improve from 0.01542
160/160 - 2s - loss: 0.0152 - accuracy: 0.1309 - val_loss: 0.0154 - val_accuracy: 0.1240
Epoch 28/100

Epoch 00028: val_loss improved from 0.01542 to 0.01530, saving model to repeat:3_Fold:4.hdf5
160/160 - 21173s - loss: 0.0152 - accuracy: 0.1347 - val_loss: 0.0153 - val_accuracy: 0.1305
Epoch 29/100

Epoch 00029: val_loss did not improve from 0.01530
160/160 - 2s - loss: 0.0150 - accuracy: 0.1360 - val_loss: 0.0153 - val_accuracy: 0.1332
Epoch 30/100

Epoch 00030: val_loss did not improve from 0.01530
160/160 - 2s - loss: 0.0149 - accur

Epoch 23/100

Epoch 00023: val_loss improved from 0.01585 to 0.01573, saving model to repeat:3_Fold:5.hdf5
160/160 - 2s - loss: 0.0159 - accuracy: 0.1179 - val_loss: 0.0157 - val_accuracy: 0.1126
Epoch 24/100

Epoch 00024: val_loss improved from 0.01573 to 0.01572, saving model to repeat:3_Fold:5.hdf5
160/160 - 2s - loss: 0.0157 - accuracy: 0.1221 - val_loss: 0.0157 - val_accuracy: 0.1202
Epoch 25/100

Epoch 00025: val_loss improved from 0.01572 to 0.01558, saving model to repeat:3_Fold:5.hdf5
160/160 - 2s - loss: 0.0155 - accuracy: 0.1271 - val_loss: 0.0156 - val_accuracy: 0.1179
Epoch 26/100

Epoch 00026: val_loss improved from 0.01558 to 0.01558, saving model to repeat:3_Fold:5.hdf5
160/160 - 2s - loss: 0.0154 - accuracy: 0.1270 - val_loss: 0.0156 - val_accuracy: 0.1155
Epoch 27/100

Epoch 00027: val_loss improved from 0.01558 to 0.01551, saving model to repeat:3_Fold:5.hdf5
160/160 - 2s - loss: 0.0153 - accuracy: 0.1301 - val_loss: 0.0155 - val_accuracy: 0.1182
Epoch 28/100

Epoch 

In [ ]:
print("Top feats length:",len(train.columns))
mod = create_model(len(train.columns))
mod.summary()

In [36]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [53]:
N_STARTS = 7
tf.random.set_seed(42)

res = train.copy()
#res.head()
ss.loc[:, train.columns] = 0
#res.loc[:, train.columns] = 0



KeyError: "None of [Index(['cp_type', 'cp_time', 'cp_dose', 'g-0', 'g-1', 'g-2', 'g-3', 'g-4',\n       'g-5', 'g-6',\n       ...\n       'c-90', 'c-91', 'c-92', 'c-93', 'c-94', 'c-95', 'c-96', 'c-97', 'c-98',\n       'c-99'],\n      dtype='object', length=875)] are in the [columns]"

In [ ]:
for seed in range(N_STARTS):
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train_targets, train_targets)):
        print(f'Fold {n}')
    
        model = create_model(len(top_feats))
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[tr][:, top_feats],
                  train_targets.values[tr],
                  validation_data=(train.values[te][:, top_feats], train_targets.values[te]),
                  epochs=35, batch_size=128,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
        model.load_weights(checkpoint_path)
        test_predict = model.predict(test.values[:, top_feats])
        val_predict = model.predict(train.values[te][:, top_feats])
        
        ss.loc[:, data_train.columns] += test_predict
        res.loc[te, data_train.columns] += val_predict
        print('')
    
ss.loc[:, data_train.columns] /= ((n+1) * N_STARTS)
res.loc[:, data_train.columns] /= N_STARTS

In [35]:
model.fit(x_train, y_train, epochs = 40, validation_split = .1, callbacks = [hist])

y_pred =  model.predict(x_test)
y_pred = np.round(y_pred[:,1])
print(metrics.accuracy_score(y_pred,y_test))

plt.plot(hist.history['acc'], color = 'red')
plt.plot(hist.history['val_acc'], color = 'blue')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

NameError: name 'model' is not defined

In [ ]:
"""Model Building"""
#Split features and labels
x = data.iloc[:, :-1].values
y = data.iloc[:,-1].values


gender_encoder = LabelEncoder()
y = gender_encoder.fit_transform(y) #Male = 1 ----- Female = 0 
                      
#Preprocessess data
scaler = StandardScaler()
x = scaler.fit_transform(x)

#Test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .1)

#Build Neural Network 

n_cols = x_train.shape[1]
y_train = to_categorical(y_train, 2)

hist = History()

model = Sequential()

model.add(Dense(1000, activation='relu', input_dim = n_cols))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))


model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 40, validation_split = .1, callbacks = [hist])

y_pred =  model.predict(x_test)
y_pred = np.round(y_pred[:,1])
print(metrics.accuracy_score(y_pred,y_test))

plt.plot(hist.history['acc'], color = 'red')
plt.plot(hist.history['val_acc'], color = 'blue')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()